<a href="https://colab.research.google.com/github/kalai2315/kalai2315/blob/main/Stock_Price_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Stock Price Prediction Project using TensorFlow**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [3]:
df=pd.read_csv("/content/BHARTIARTL (1).csv",parse_dates=True,index_col='Date')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df['Close'].plot()

In [10]:
len(df)

4774

In [12]:
airtel_close=df['Close']
airtel_close.shape

(4774,)

In [15]:
airtel_close.reset_index(drop=True,inplace=True)
airtel_close.shape

(4774, 1)

In [13]:
airtel_close=pd.DataFrame(airtel_close)

In [ ]:
airtel_close

In [16]:
train=airtel_close.iloc[:4750]
test=airtel_close.iloc[4751:]

In [17]:
from sklearn.preprocessing import MinMaxScaler

In [18]:
scaler=MinMaxScaler()

In [19]:
scaler.fit(train)

MinMaxScaler()

In [20]:
scaled_train=scaler.transform(train)
scaled_test=scaler.transform(test)

In [21]:
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

creating the model

In [22]:
test.shape

(23, 1)

In [ ]:
help(TimeseriesGenerator)

In [24]:
length=23
batch_size=32

generator=TimeseriesGenerator(scaled_train,scaled_train,
                              length=length,batch_size=batch_size)

#will create an object with train - length, and last 23 will be predicted and validated

In [25]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,SimpleRNN,LSTM, Dropout

In [26]:
n_features=1

In [27]:
model=Sequential()
model.add(SimpleRNN(30,input_shape=(length,n_features)))

model.add(Dropout(rate=0.2))
model.add(Dense(1))
model.compile(optimizer='adam',loss='mse')

In [ ]:
model.summary()

In [ ]:
model.fit_generator(generator,epochs=30,shuffle=False )

In [64]:
losses=pd.DataFrame(model.history.history)

In [ ]:
losses.plot()

Evaluate the TestSet


In [66]:
 first_batch=scaled_train[-length:]

In [ ]:
first_batch

In [68]:
first_batch=first_batch.reshape((1,length,1))

In [ ]:
first_batch

In [72]:
model.predict(first_batch)

1/1 [==============================] - 0s 15ms/step


array([[0.4598553]], dtype=float32)

In [ ]:
test_predictions=[]
first_batch=scaled_train[-length:]
cur_batch=first_batch.reshape((1,length,n_features))
for i in range(len(test)):
  cur_pred=model.predict(cur_batch)[0]
  test_predictions.append(cur_pred)
  cur_batch=np.append(cur_batch[:,1:,:],[[cur_pred]],axis=1)

In [ ]:
test_predictions

In [76]:
true_predictions=scaler.inverse_transform(test_predictions)

In [ ]:
true_predictions

In [78]:
test['predictions']=true_predictions

In [ ]:
test

In [ ]:
test.plot(figsize=(12,8))
airtel_close.plot()

Create for Full Model

In [81]:
full_scaler=MinMaxScaler()
scaled_full_data=full_scaler.fit_transform(airtel_close)

In [82]:
generator=TimeseriesGenerator(scaled_full_data,scaled_full_data,
                              length=length,batch_size=32)

In [83]:
model=Sequential()
model.add(SimpleRNN(30,input_shape=(length,n_features)))

model.add(Dropout(rate=0.2))
model.add(Dense(1))
model.compile(optimizer='adam',loss='mse')

In [ ]:
model.fit_generator(generator,epochs=30,shuffle=False )

In [85]:
full_model_losses=pd.DataFrame(model.history.history)

In [ ]:
full_model_losses.plot()

In [ ]:
fore_cast=[]
first_batch=scaled_train[-length:]
cur_batch=first_batch.reshape((1,length,n_features))
for i in range(len(test)):
  cur_pred=model.predict(cur_batch)[0]
  fore_cast.append(cur_pred)
  cur_batch=np.append(cur_batch[:,1:,:],[[cur_pred]],axis=1)

In [89]:
fore_cast=scaler.inverse_transform(fore_cast)
fore_cast

array([[530.96181123],
       [529.83061551],
       [529.41574826],
       [530.283186  ],
       [530.75064021],
       [530.18298431],
       [529.77885178],
       [529.25808619],
       [528.61838255],
       [527.96033769],
       [527.39666615],
       [526.48770546],
       [526.27248423],
       [525.1938096 ],
       [524.72421504],
       [524.39937454],
       [523.4736532 ],
       [523.0475573 ],
       [522.69492507],
       [522.02035006],
       [521.58022658],
       [521.22927371],
       [520.83880653]])

In [90]:
fore_cast.flatten()

array([530.96181123, 529.83061551, 529.41574826, 530.283186  ,
       530.75064021, 530.18298431, 529.77885178, 529.25808619,
       528.61838255, 527.96033769, 527.39666615, 526.48770546,
       526.27248423, 525.1938096 , 524.72421504, 524.39937454,
       523.4736532 , 523.0475573 , 522.69492507, 522.02035006,
       521.58022658, 521.22927371, 520.83880653])

In [91]:
df_array=np.array(airtel_close)
df_array

array([[ 44.35],
       [ 41.7 ],
       [ 41.25],
       ...,
       [544.3 ],
       [540.85],
       [536.75]])

In [94]:
full_data=pd.DataFrame(np.concatenate((df_array.flatten(),fore_cast.flatten())))

In [ ]:
full_data.plot();

LSTM MOdel

In [97]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop=EarlyStopping(monitor='val_loss',patience=5)

In [99]:
length

23

In [101]:
scaled_test.shape

(23, 1)

In [103]:
length=22
generator=TimeseriesGenerator(scaled_train,scaled_train,
                              length=length,batch_size=32)

validation_generator=TimeseriesGenerator(scaled_test,scaled_test,
                              length=length,batch_size=32)

In [ ]:
model=Sequential()
model.add(LSTM(30,input_shape=(length,n_features)))

model.add(Dropout(rate=0.2))
model.add(Dense(1))
model.compile(optimizer='adam',loss='mse')

In [104]:
model.fit_generator(generator,epochs=30,
                    validation_data=validation_generator,
                    callbacks=[early_stop],shuffle=False )

Epoch 1/30
148/148 [==============================] - 1s 5ms/step - loss: 5.7325e-04 - val_loss: 1.5783e-04
Epoch 2/30
148/148 [==============================] - 1s 3ms/step - loss: 5.2307e-04 - val_loss: 1.1904e-06
Epoch 3/30
148/148 [==============================] - 1s 3ms/step - loss: 6.0382e-04 - val_loss: 2.2817e-04
Epoch 4/30
148/148 [==============================] - 1s 3ms/step - loss: 4.7502e-04 - val_loss: 1.9903e-05


In [105]:
losses=pd.DataFrame(model.history.history)

In [ ]:
losses.plot()

In [107]:
test_predictions=[]
first_batch=scaled_train[-length:]
cur_batch=first_batch.reshape((1,length,n_features))
for i in range(len(test)):
  #get prediction 1 time stam ahead([0] is for grabing just the number instead of array)
  cur_pred=model.predict(cur_batch)[0]
  test_predictions.append(cur_pred)
  #update batch to now include prediction and drop first value
  cur_batch=np.append(cur_batch[:,1:,:],[[cur_pred]],axis=1)

1/1 [==============================] - 0s 17ms/step


In [ ]:
true_predictions=scaler.inverse_transform(test_predictions)
test['LSTM_predictions']=true_predictions
test.plot(figsize=(12,8))

In [ ]:
fore_cast=[]
first_batch=scaled_train[-length:]
cur_batch=first_batch.reshape((1,length,n_features))
for i in range(22):
  #get prediction 1 time stam ahead([0] is for grabing just the number instead of array)
  cur_pred=model.predict(cur_batch)[0]
  fore_cast.append(cur_pred)
  #update batch to now include prediction and drop first value
  cur_batch=np.append(cur_batch[:,1:,:],[[cur_pred]],axis=1)

In [115]:
fore_cast=scaler.inverse_transform(fore_cast)


In [116]:
full_data=pd.DataFrame(np.concatenate((df_array.flatten(),fore_cast.flatten())))

In [ ]:
full_data.plot();

In [ ]:
fore_cast

In [ ]:
full_data.tail(30)

FB prophet

In [ ]:
df.reset_index(inplace=True)

In [ ]:
data=df[['Date','close']]

In [ ]:
data.head()

In [ ]:
data=data.rename(columns={"Date":"ds","close":"y"})

In [ ]:
data.head()

In [ ]:
from fbprophet import Prophet
m=Prophet(changepoint_prior_scale=0.15,daily_seasonality=True) #the prophet cl;ass model
m.fit(data)

In [ ]:
future=m.make_future_dataframe(periods=21)#specify the number of days in future
prediction=m.predict(future)
m.plot(prediction)
plt.title("Prediction of google stock price using the prophet")
plt.xlabel("Date")
plt.ylabel("Close Stock Price")
plt.show()

In [ ]:
m.plot_components(prediction)
plt.show()